In [1]:
# Transfer learning: Using a pretrained network on images not in training 
# set .
# ImageNet dataset: 1 million labeled images in 1000 categories.
# To classify cat and dog photos

In [2]:
# In neural networks there is tradeoff between spped and accuracy, ie
# larger or smaller networks
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models

In [3]:
data_dir = 'Cat_Dog_data'

# Most of pre-trained models require input to be 224*224 images, 
# We also need to match normalization used when models were trained.
# Each color channel was normalozed separately, the means are [0.485,
# 0.456, 0.406] ans standard deviations are [0.229,0.224,0.225]

train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                               transforms.Normalize((0.485, 0.456, 0.406),
                              (0.229, 0.224, 0.225))])

test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                               transforms.Normalize((0.485, 0.456, 0.406),
                              (0.229, 0.224, 0.225))])

# transforms.RandomHorizontalFlip() works on PIL.Images, not 
# torch.Tensor. So apply transforms.ToTensor() after
# transforms.RandomHorizontalFlip().



# Transforms:
"""transforms.Compose clubs all transforms provided to it. So, all 
transforms in transforms.Compose are applied to input one by one.

Train transforms:
1) transforms.RandomResizedCrop(224): This extract a patch of size 
(224, 224) from input image randomly. So, it might pick this path from
topleft, bottomright or anywhere in between. So, you are doing data 
augmentation in this part. Also, changing this value won't play nice 
with the fully-connected layers in your model, so not advised to 
change this.
2) transforms.RandomHorizontalFlip(): Once we have our image of size 
(224, 224), we can choose to flip it. This is another part of data 
augmentation.
3) transforms.ToTensor(): This converts input image to PyTorch tensor.
4) transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]):
This is input data scaling and these values (mean and std) must have 
been precomputed for your dataset. Changing these values is also not 
advised.

Validation transforms:
1) transforms.Resize(256): First your input image is resized to be of 
size (256, 256).
2) transforms.CentreCrop(224): Crops the center part of the image of 
shape (224, 224).
Rest are the same as train"""


train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)


In [4]:
# https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
# print(trainloader)

# Downloading the pre-trained network densenet
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
# The classifer part is single fully-connected layer, which was trained
# on ImageNet dataset, so it won't work on our problem, and we have to 
# retrain classifier, but features will work perfectly on their own.

# freeze our feature parameters
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
   ('fc1', nn.Linear(1024, 500)),
    ('relu', nn.ReLU()),
    ('fc2',nn.Linear(500,2)),
    ('output', nn.LogSoftmax(dim=1))
]))
model.classifier = classifier

In [6]:
"""Training such deep nn with cpu takes very long time, Gpu can be used
to do calculations. Linear computations are done parallel in GPU so 100x
increased training speeds"""
# In pytorch, model parameters and other tensors can be moved to GPU
# using model.cuda. and can be moved back with model.cpu()
# model.cuda()

# images.cuda()

# model.cpu(), images.cpu()

'Training such deep nn with cpu takes very long time, Gpu can be used\nto do calculations. Linear computations are done parallel in GPU so 100x\nincreased training speeds'

In [7]:
import time
for cuda in [True,False]: #[False, True]
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    if cuda:
        model.cuda()
    else:
        model.cpu()
        
    for ii,(inputs,labels) in enumerate(trainloader):
        print(ii)
        inputs,labels = Variable(inputs), Variable(labels)
        
        if cuda:
            inputs,labels = inputs.cuda(), labels.cuda()
        
        start = time.time()
        
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if ii==3:
            break
            
    print(f"CUDA = {cuda}; Time per batch: {(time.time()- start)/3:.3f} seconds")        

RuntimeError: cuda runtime error (804) : forward compatibility was attempted on non supported HW at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/THC/THCGeneral.cpp:50

In [8]:
# conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

In [9]:
# conda update -n base conda

Solving environment: | 
  - https://repo.continuum.io/pkgs/main/linux-64/get_terminal_size-1.0.0-haa9412d_0.tar.bz2/linux-64::get_terminal_size-1.0.0-haa9412d_0, https://repo.continuum.io/pkgs/main/linux-64/jbig-2.1-hdba287a_0.tar.bz2/linux-64::jbig-2.1-hdba287a_0, https://repo.continuum.io/pkgs/main/linux-64/linux-64::asn1crypto-1.3.0-py37_0, https://repo.continuum.io/pkgs/main/linux-64/linux-64::six-1.14.0-py37_0
  - defaults/linux-64::get_terminal_size-1.0.0-haa9412d_0, https://repo.continuum.io/pkgs/main/linux-64/jbig-2.1-hdba287a_0.tar.bz2/linux-64::jbig-2.1-hdba287a_0, https://repo.continuum.io/pkgs/main/linux-64/linux-64::asn1crypto-1.3.0-py37_0, https://repo.continuum.io/pkgs/main/linux-64/linux-64::six-1.14.0-py37_0
  - defaults/linux-64::jbig-2.1-hdba287a_0, https://repo.continuum.io/pkgs/main/linux-64/get_terminal_size-1.0.0-haa9412d_0.tar.bz2/linux-64::get_terminal_size-1.0.0-haa9412d_0, https://repo.continuum.io/pkgs/main/linux-64/linux-64::asn1crypto-1.3.0-py37_0, https:/

# Resnet Model

In [10]:
# Checking if gpu is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained = True)
# fc - fully connected
# model

# Turn of gradients for model
for param in model.parameters():
    param.requires_grad = False
    
# New classifier for cat/dog
classifier = nn.Sequential(nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.2),
                          nn.Linear(512, 2),
                          nn.LogSoftmax(dim=1))
model.fc = classifier
# model

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.03)
model.to(device);

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    for images, labels in trainloader:
        steps+=1
        
        images,labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every ==0:
            model.eval()
            test_loss = 0
            accuracy = 0
            for images, labels in testloader:
                
                images,labels = images.to(device), labels.to(device)
                
                logps = model(images)
                loss = criterion(logps, labels)
                test_loss += loss.item()
                
                # Calculate accuracy
                ps = torch.exp(logps)
                top_ps, top_class = ps.topk(1, dim=1)
                equality = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
                
            print(f"Epoch {epoch+1}/{epochs}.. "
                 f"Train loss: {running_loss/print_every:.3f}.. "
                 f"Test loss: {test_loss/len(testloader):.3f}.. "
                 f"Test accuracy: {accuracy/len(testloader):.3f}") 
            
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 63.297.. Test loss: 102.666.. Test accuracy: 0.494
Epoch 1/1.. Train loss: 41.127.. Test loss: 9.444.. Test accuracy: 0.646
Epoch 1/1.. Train loss: 9.291.. Test loss: 2.884.. Test accuracy: 0.812
Epoch 1/1.. Train loss: 3.547.. Test loss: 0.462.. Test accuracy: 0.945
Epoch 1/1.. Train loss: 2.452.. Test loss: 2.152.. Test accuracy: 0.877
Epoch 1/1.. Train loss: 1.476.. Test loss: 0.827.. Test accuracy: 0.914
Epoch 1/1.. Train loss: 0.566.. Test loss: 0.449.. Test accuracy: 0.933
Epoch 1/1.. Train loss: 0.355.. Test loss: 0.206.. Test accuracy: 0.948
Epoch 1/1.. Train loss: 0.266.. Test loss: 0.238.. Test accuracy: 0.926
Epoch 1/1.. Train loss: 0.212.. Test loss: 0.143.. Test accuracy: 0.938


In [ ]:
# Errors chances
"""In general, you'll want to check that the tensors going through your
model and other code are the correct shapes. Make use of the .shape 
method during debugging and development."""

"""Make sure you're clearing the gradients in the training loop with 
optimizer.zero_grad(). If you're doing a validation loop, be sure to 
set the network to evaluation mode with model.eval(), then back to 
training mode with model.train()."""

"""Cuda error: RuntimeError: Expected object of type torch.FloatTensor 
but found type torch.cuda.FloatTensor for argument #1 ‘mat1’

You'll notice second type is torch.cuda.FloatTensor, this means it's a 
tensor that has been moved to the GPU. It's expecting a tensor with 
type torch.FloatTensor, no .cuda there, which means the tensor should
be on the CPU. PyTorch can only perform operations on tensors that are
on the same device, so either both CPU or both GPU. If you're trying to
run your network on the GPU, check to make sure you've moved the model 
and all necessary tensors to the GPU with .to(device) where device is 
either "cuda" or "cpu"."""